## Libraries Required

In [411]:
import numpy as np
import pandas as pd
import csv
import math
import sklearn.utils as sk

## Human Observed Data Preprocessing

In [ ]:
'''with open("HumanObserved-Features-Data.csv", "r") as f: #Read features csv
    x = pd.read_csv(f)
    
with open("diffn_pairs.csv", "r") as g: # Read Different pairs csv
    y = pd.read_csv(g)
    
with open("same_pairs.csv", "r") as g:  #Read same pairs csv
    z = pd.read_csv(g)
    
y = y.reindex(np.random.permutation(y.index))  # Randomize the csv file
y1 = y[0:2500]
Raw = pd.concat([y1,z],ignore_index = True)
Raw = Raw.reindex(np.random.permutation(Raw.index))

#Concatenation
df_merge = pd.merge(Raw,x,left_on='img_id_A',right_on='img_id')
df_merge=pd.merge(df_merge,x,left_on='img_id_B',right_on='img_id') 
df_merge=df_merge.drop(df_merge.columns[[0,1,3,4,14,15]],axis=1)
df_merge = df_merge.loc[:,(df_merge!=0).any(axis=0)] #Remove zero variance features
df_merge.to_csv('HumanFeatureData.csv')

#Subtraction
dfA=pd.merge(Raw,x,left_on='img_id_A',right_on='img_id')
dfB=pd.merge(Raw,x,left_on='img_id_B',right_on='img_id')
dfA=dfA.drop(dfA.columns[[0,1,2,3,4]],axis=1)
dfB=dfB.drop(dfB.columns[[0,1,2,3,4]],axis=1)
dfSub=abs(dfA-dfB)
dfSub=Raw.merge(dfSub,how='outer',left_index=True, right_index=True)
dfSub=dfSub.drop(dfSub.columns[[0,1]],axis=1)
df_Sub = df_Sub.loc[:,(df_Sub!=0).any(axis=0)] #Remove zero variance features
dfSub.to_csv('HumanFeatureDataSub.csv')'''

## GSC Data Preprocessing

In [ ]:
'''with open("GSC-Features.csv", "r") as f:  #Read features csv
    x = pd.read_csv(f)
    
with open("diffn_pairs_gsc.csv", "r") as g:  # Read Different pairs csv
    y = pd.read_csv(g)
    
with open("same_pairs_gsc.csv", "r") as g: #Read same pairs csv
    z = pd.read_csv(g)
    
y = y.reindex(np.random.permutation(y.index))  # Randomize the two csv files
z = z.reindex(np.random.permutation(z.index))
y1 = y[0:2000]
z1 = z[0:1000]  
Raw = pd.concat([y1,z1],ignore_index = True)  #Concatenate the two csv files
Raw = Raw.reindex(np.random.permutation(Raw.index))  #Randomize the csv file

#Concatenation
df_merge = pd.merge(Raw,x,left_on='img_id_A',right_on='img_id')
df_merge=pd.merge(df_merge,x,left_on='img_id_B',right_on='img_id') 
df_merge=df_merge.drop(df_merge.columns[[0,1,3,516]],axis=1)
df_merge = df_merge.loc[:,(df_merge!=0).any(axis=0)] #Remove zero variance features
df_merge.to_csv('GSCFeatureData.csv')

#Subtraction
dfA=pd.merge(Raw,x,left_on='img_id_A',right_on='img_id')
dfB=pd.merge(Raw,x,left_on='img_id_B',right_on='img_id')
dfA=dfA.drop(dfA.columns[[0,1,2,3]],axis=1)
dfB=dfB.drop(dfB.columns[[0,1,2,3]],axis=1)
dfSub=abs(dfA-dfB)
dfSub=Raw.merge(dfSub,how='outer',left_index=True, right_index=True)
dfSub=dfSub.drop(dfSub.columns[[0,1]],axis=1)
dfSub = dfSub.loc[:,(dfSub!=0).any(axis=0)]  #Remove zero variance features
dfSub.to_csv('GSCFeatureDataSub.csv')'''

## Initial Variables

In [389]:
TrainingPercent = 80 #Training split
ValidationPercent = 10 #Validation split
TestPercent = 10 #Test split
LearningRate = 0.001

In [412]:
file = pd.read_csv('GSCFeatureDataSub.csv')
lenc = len(file.columns) #To get the number of features
lenc

507

## Functions to Generate the Data

In [413]:
def GenerateRawData(file_path): #Generate the Raw Feature Matrix and Target Vector
    data_target = []
    data_input = []
    x = pd.read_csv(file_path)
    
    data_target = x['target'].tolist()

    data_input = x.iloc[:,2:lenc]
    
    data = np.array(data_input).T
    target = data_target

    return data,target

def GenerateTrainingTarget(rawTraining,TrainingPercent = 80):  #Splitting data to get Training target data
    TrainingLen = int(math.ceil(len(rawTraining)*(TrainingPercent*0.01)))
    t           = rawTraining[:TrainingLen]
    return t

def GenerateTrainingDataMatrix(rawData, TrainingPercent = 80): #Splitting data to get Training feature data
    T_len = int(math.ceil(len(rawData[0])*0.01*TrainingPercent))
    d2 = rawData[:,0:T_len]
    return d2

def GenerateValData(rawData, ValPercent, TrainingCount): #Splitting data to get Validation feature data
    valSize = int(math.ceil(len(rawData[0])*ValPercent*0.01))
    V_End = TrainingCount + valSize
    dataMatrix = rawData[:,TrainingCount+1:V_End]  
    return dataMatrix

def GenerateValTargetVector(rawData, ValPercent, TrainingCount): #Splitting data to get Validation target data
    valSize = int(math.ceil(len(rawData)*ValPercent*0.01))
    V_End = TrainingCount + valSize
    t =rawData[TrainingCount+1:V_End]
    return t

def Sigmoid(value):  #Function to Calculate the sigmoid of a value
    return 1 / (1 + np.exp(-value))

def predictY(Data,W):  #Function to predict the Target Value
    val = np.dot(W.T,Data)
    return Sigmoid(val)


## Generate Raw Data

In [425]:
#RawData,RawTarget = GenerateRawData('HumanFeatureData.csv') #Function call to make Feature Matrix and Target Vector
#RawData,RawTarget = GenerateRawData('HumanFeatureDataSub.csv')
RawData,RawTarget = GenerateRawData('GSCFeatureData.csv')
#RawData,RawTarget = GenerateRawData('GSCFeatureDataSub.csv')

## Training Data

In [426]:
TrainingTarget = np.array(GenerateTrainingTarget(RawTarget,TrainingPercent)) #Prepare the target vector for training
#dataset
TrainingData   = GenerateTrainingDataMatrix(RawData,TrainingPercent) #Prepare the Feature matrix for training dataset
print(TrainingTarget.shape)
print(TrainingData.shape)
    

(2400,)
(505, 2400)


## Validation Data

In [427]:
ValDataAct = np.array(GenerateValTargetVector(RawTarget,ValidationPercent, (len(TrainingTarget)))) #Prepare the 
#Validation target vector
ValData    = GenerateValData(RawData,ValidationPercent, (len(TrainingTarget)))# Prepare the feature matrix for 
#Validation Dataset
print(ValDataAct.shape)
print(ValData.shape)

(299,)
(505, 299)


## Test Data

In [428]:
TestDataAct = np.array(GenerateValTargetVector(RawTarget,TestPercent, (len(TrainingTarget)+len(ValDataAct))))#Prepare
#the Target vector for testing dataset
TestData = GenerateValData(RawData,TestPercent, (len(TrainingTarget)+len(ValDataAct)))# Prepare the feature matrix 
#for Testing datasewt
print(TestDataAct.shape)
print(TestData.shape)

(299,)
(505, 299)


## Run the Model

In [429]:
 W = np.ones((lenc-2,1)) #Initialize the weights

for itera in range(0,1000):
    PredictY = predictY(TrainingData,W)  #Predict target values
    Sub = PredictY - TrainingTarget   #This is to calculate the Gradient.
    Delta_w = np.dot(TrainingData,Sub.T)
    Delta_w = Delta_w/len(TrainingData)
    Delta_w = Delta_w*LearningRate  #Gradient Calculated
    W = W - Delta_w  #Updating the weights

## Training Accuracy

In [430]:
#Calculate Training Accuracy
TR_Y = Sigmoid(np.dot(np.transpose(W),TrainingData)) 
Predicted = np.round(TR_Y)
TrAccuracy = (TrainingTarget == Predicted).sum()
TrAccuracy = TrAccuracy/len(TrainingTarget)
print(TrAccuracy)   

0.61125


## Run for Validation

In [431]:
for i in range(0,1000):  #Run Iterations for validation
    PredictY = predictY(ValData,W)
    Sub = PredictY - ValDataAct
    Delta_w = np.dot(ValData,Sub.T)
    Delta_w = Delta_w/len(ValData)
    Delta_w = Delta_w*LearningRate
    W = W - Delta_w

## Validation Accuracy

In [432]:
#Calculate Validation Accuracy
V_Y = Sigmoid(np.dot(np.transpose(W),ValData)) 
Predicted = np.round(V_Y)
VAccuracy = (ValDataAct == Predicted).sum()
VAccuracy = VAccuracy/len(ValDataAct)
print(VAccuracy)

0.5183946488294314


## Testing Accuracy

In [433]:
#Calculate Testing Accuracy
T_Y = Sigmoid(np.dot(np.transpose(W),TestData)) 
Predicted = np.round(T_Y)
TAccuracy = (TestDataAct == Predicted).sum()
TAccuracy = TAccuracy/len(TestDataAct)
print(TAccuracy)

0.4682274247491639
